## Fixed-Income Pricing Model Delovpment And Validation 

### Pricing Model: DCF | Asset Class: Fixed Income | Finacial Product: US Treasury Bond

#### Import Liabaries

In [3]:
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from datetime import date, timedelta

In [5]:
dayCountMapping = {'1 Mo': 30, '2 Mo': 60, '3 Mo': 90, '4 Mo': 120, '6 Mo': 180, '1 Yr': 360, 
                   '2 Yr': 720, '3 Yr': 1080, '5 Yr': 1800, '7 Yr': 2520, '10 Yr': 3600, '20 Yr': 7200, '30 Yr': 10800}

#### Instrument Trade Data

In [7]:
trade_data = pd.read_csv("C:\\Users\\dines\\Desktop\\Market Risk Project\\2Y US Treasury Note_91282CKB6\\TREASURY AUCTION RESULTS for 91282CKB6.csv",index_col="CUSIP")

In [9]:
trade_data

,Security Type,Security Term,Auction Date,Issue Date,Maturity Date,Price per $100
CUSIP,,,,,,
91282CKB6,Note,2-Year,02/26/2024,02/29/2024,02/28/2026,99.875391


#### Insturment Features

In [11]:
faceValue = 100
cuponRate = 0.04625
maturityPeriod = 2
frequency = 2
auctionDate = pd.to_datetime(trade_data['Auction Date'].values[-1]).date()
issueDate = pd.to_datetime(trade_data['Issue Date'].values[-1]).date()
maturityDate = pd.to_datetime(trade_data['Maturity Date'].values[-1]).date()
issuePrice = trade_data['Price per $100'].values[-1]

In [13]:
valuationDate = auctionDate

In [17]:
issueDate

datetime.date(2024, 2, 29)

#### Coupon Dates

In [21]:
Y, M, D = issueDate.year, issueDate.month, issueDate.day

cuponDates = []
for i in range(1,(maturityPeriod*frequency) + 1):
    try:
        if i % 2 != 0:
            cuponDates.append(date(Y, M+6, D))
        else:
            cuponDates.append(date(Y+1, M, D))
    except ValueError:
        cuponDates.append(date(Y+1,M,D-1))
print(cuponDates)

[datetime.date(2024, 8, 29), datetime.date(2025, 2, 28), datetime.date(2024, 8, 29), datetime.date(2025, 2, 28)]


In [23]:
cuponDays = [180 * i for i in range(1,(maturityPeriod*frequency) + 1)]
print(cuponDays)

[180, 360, 540, 720]


## Market Data Preparation

#### Intrest Rate

In [25]:
class InterestRates:

    def __init__(self, Y):
        self.Y = Y

    def USTreasury(self, ratesCurve=False, COBDate=None):
        USTreasuryData = pd.read_csv(f'https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/{self.Y}/all?type=daily_treasury_yield_curve&field_tdr_date_value={self.Y}&page&_format=csv', index_col='Date').iloc[::-1]  
        USTreasuryData.index = pd.to_datetime(USTreasuryData.index)
        
        if ratesCurve == True:
            if COBDate == None:
                return USTreasuryData[USTreasuryData.index == USTreasuryData.index.max()]
            else:
                return USTreasuryData[USTreasuryData.index == COBDate]
        else:
            return USTreasuryData

In [27]:
spotRates = InterestRates(Y=valuationDate.year).USTreasury(ratesCurve=True, COBDate=pd.to_datetime(valuationDate))
spotRates = pd.DataFrame(data=spotRates.values, index=spotRates.index, columns=dayCountMapping.values())

In [28]:
spotRates

,30,60,90,120,180,360,720,1080,1800,2520,3600,7200,10800
Date,,,,,,,,,,,,,
2024-02-26,5.5,5.52,5.47,5.47,5.34,5.03,4.69,4.48,4.29,4.32,4.28,4.53,4.4


####  Interst Rate Determination

In [31]:
tenors , interestRates = spotRates.columns/360 , spotRates.values[0]/100
print(tenors, interestRates)

Index([0.08333333333333333, 0.16666666666666666,                0.25,
        0.3333333333333333,                 0.5,                 1.0,
                       2.0,                 3.0,                 5.0,
                       7.0,                10.0,                20.0,
                      30.0],
      dtype='float64') [0.055  0.0552 0.0547 0.0547 0.0534 0.0503 0.0469 0.0448 0.0429 0.0432
 0.0428 0.0453 0.044 ]


In [33]:
interpfunction = interpolate.CubicSpline(x= tenors, y= interestRates)
tenorsInterp = [t/360 for t in cuponDays]
interpolatedRates = interpfunction(tenorsInterp)
print(interpolatedRates)

[0.0534     0.0503     0.04842681 0.0469    ]


### Pricing Model

#### Discount Cashflow (DCF) Approach

In [35]:
futurevalueCF = [faceValue*(cuponRate/frequency) for t in range(1,(maturityPeriod*frequency)+1)]
futurevalueCF[-1] += faceValue
print(futurevalueCF)

[2.3125, 2.3125, 2.3125, 102.3125]


In [37]:
annualizedRate = [(1+i/frequency)**frequency -1 for i in interpolatedRates]
print(annualizedRate)

[0.05411288999999986, 0.05093252249999991, 0.04901309396537723, 0.04744990249999992]


In [39]:
presentValueCF = [futurevalueCF[i]/(1+interpolatedRates[i]/frequency) ** (i+1) for i in range(maturityPeriod*frequency)]
bondPrice = sum(presentValueCF)
print(bondPrice,presentValueCF)

99.8579894471851 [2.2523619363007694, 2.2004267167400373, 2.1523375259258706, 93.25286326821842]


#### Model Error

In [41]:
print('Model Price:',bondPrice)
print('Market Price:',issuePrice)

priceDiff = bondPrice-issuePrice
print('Price Difference:', round(priceDiff,4))

Model Price: 99.8579894471851
Market Price: 99.875391
Price Difference: -0.0174


In [46]:
pricingResult =pd.Series({"Model Price":bondPrice,"Issue Price":issuePrice,"Price Difference":priceDiff})

In [48]:
pricingResult.to_csv("C:\\Users\\dines\\Desktop\\Market Risk Project\\2Y US Treasury Note_91282CKB6\\Pricing RESULTS for 91282CKB6.csv")